# Animelist: GRU

## Library Installation

In [1]:
!pip install recommenders cornac==2.3.0 ranx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 19.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.4/25.4 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.3/355.3 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 119.6 MB/s

In [2]:
!add-apt-repository ppa:ubuntu-toolchain-r/test
!apt-get update
!apt-get install --only-upgrade libstdc++6

PPA publishes dbgsym, you may need to include 'main/debug' component
Repository: 'deb https://ppa.launchpadcontent.net/ubuntu-toolchain-r/test/ubuntu/ jammy main'
Description:
Toolchain test builds; see https://wiki.ubuntu.com/ToolChain

More info: https://launchpad.net/~ubuntu-toolchain-r/+archive/ubuntu/test
Adding repository.
Press [ENTER] to continue or Ctrl-c to cancel.
Adding deb entry to /etc/apt/sources.list.d/ubuntu-toolchain-r-ubuntu-test-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/ubuntu-toolchain-r-ubuntu-test-jammy.list
Adding key to /etc/apt/trusted.gpg.d/ubuntu-toolchain-r-ubuntu-test.gpg with fingerprint C8EC952E2A0E1FBDC5090F6A2C277A0A352154E5
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,607 kB]
Get:4 http://secur

## Preparation

In [1]:
import pandas as pd
import sys
import cornac
import pickle

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import (
    rmse,
    mae,
    rsquared,
    exp_var,
    map_at_k,
    ndcg_at_k,
    precision_at_k,
    recall_at_k,
    get_top_k_items,
)

from recommenders.utils.notebook_utils import store_metadata
from recommenders.models.cornac.cornac_utils import predict_ranking

from ranx import Qrels, Run, evaluate

print(f"System version: {sys.version}")
print(f"Cornac version: {cornac.__version__}")

System version: 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]
Cornac version: 2.3.0


In [2]:
DATA_PATH = '/content/drive/MyDrive/Final Project/Codes/animelist-goodbooks-recommendation/animelist/data_sample_split/'

In [3]:
train_data = pd.read_csv(f'{DATA_PATH}/data_train_full.tsv', sep='\t')
test_data = pd.read_csv(f'{DATA_PATH}/data_test.tsv', sep='\t')

In [4]:
# top k items to recommend
TOP_K = 20
NUM_EPOCHS = 20
SEED=100

In [5]:
train_data['updated_at'] = pd.to_datetime(train_data["updated_at"], utc=True).astype(int) // 10**9
test_data['updated_at'] = pd.to_datetime(test_data["updated_at"], utc=True).astype(int) // 10**9

In [ ]:
from cornac.eval_methods import NextItemEvaluation

next_item_eval = NextItemEvaluation.from_splits(
    train_data=list(train_data[['user_id', 'anime_id', 'updated_at']].itertuples(index=False)),
    test_data=list(test_data[['user_id', 'anime_id', 'updated_at']].itertuples(index=False)),
    exclude_unknowns=True,
    verbose=True,
    fmt="SIT",
)

rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 1
Number of items = 1000
Number of sessions = 10000
---
Test data:
Number of users = 1
Number of items = 1000
Number of sessions = 10000
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 1
Total items = 1000
Total sessions = 20000


In [ ]:
from collections import OrderedDict, defaultdict
import numpy as np
from tqdm.notebook import tqdm

def ranking_eval(
    model,
    train_set,
    test_set,
    exclude_unknowns=True,
    mode="last",
    verbose=False,
):

    rankings = []
    scores = []
    user_sessions = defaultdict(list)
    session_ids = []
    for [sid], [mapped_ids], [session_items] in tqdm(
        test_set.si_iter(batch_size=1, shuffle=False),
        total=len(test_set.sessions)
    ):

        if len(session_items) < 2:  # exclude all session with size smaller than 2
            continue
        user_idx = test_set.uir_tuple[0][mapped_ids[0]]
        session_ids.append(sid)

        start_pos = 1 if mode == "next" else len(session_items) - 1
        for test_pos in range(start_pos, len(session_items), 1):
            test_pos_items = session_items[test_pos]

            # binary mask for ground-truth positive items
            u_gt_pos_mask = np.zeros(test_set.num_items, dtype="int")
            u_gt_pos_mask[test_pos_items] = 1

            # binary mask for ground-truth negative items, removing all positive items
            u_gt_neg_mask = np.ones(test_set.num_items, dtype="int")
            u_gt_neg_mask[test_pos_items] = 0

            # filter items being considered for evaluation
            if exclude_unknowns:
                u_gt_pos_mask = u_gt_pos_mask[: train_set.num_items]
                u_gt_neg_mask = u_gt_neg_mask[: train_set.num_items]

            u_gt_pos_items = np.nonzero(u_gt_pos_mask)[0]
            u_gt_neg_items = np.nonzero(u_gt_neg_mask)[0]
            item_indices = np.nonzero(u_gt_pos_mask + u_gt_neg_mask)[0]


            item_rank, item_scores = model.rank(
                user_idx,
                item_indices,
                history_items=session_items[:test_pos],
                history_mapped_ids=mapped_ids[:test_pos],
                sessions=test_set.sessions,
                session_indices=test_set.session_indices,
                extra_data=test_set.extra_data,
            )
            item_scores = item_scores[item_rank]
            item_rank = [key for value in item_rank for key, val in train_set.iid_map.items() if val == value]

            rankings.append(item_rank)
            scores.append(item_scores)

    return rankings, scores

## Default Model

In [ ]:
 gru = cornac.models.GRU4Rec(
        n_epochs=NUM_EPOCHS,
        device="cuda",
        verbose=True,
        batch_size=512,
        seed=SEED,
    )

In [ ]:
with Timer() as t:
    gru.fit(next_item_eval.train_set)
print("Took {} seconds for training.".format(t))

  0%|          | 0/20 [00:00<?, ?it/s]

Took 187.2571 seconds for training.


In [ ]:
gru.transform(next_item_eval.test_set)

In [ ]:
gru_ranking, gru_scores = ranking_eval(
    gru,
    next_item_eval.train_set,
    next_item_eval.test_set,
    exclude_unknowns=True,
    mode="last",
    verbose=False,
)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
users = []
item = list(gru.train_set.iid_map.keys())
for uid, user_idx in gru.train_set.sid_map.items():
    user = [uid] * len(item)
    users.extend(user)

In [ ]:
import itertools

flattened_ranking = list(itertools.chain(*gru_ranking))
flattened_score= list(itertools.chain(*gru_scores))

In [ ]:
len(users), len(flattened_ranking), len(flattened_score)

(10000000, 10000000, 10000000)

In [ ]:
df_gru_predictions = pd.DataFrame({'user_id':users, 'anime_id':flattened_ranking, 'prediction': flattened_score})

In [ ]:
ranking_metrics_gru = {
    'Precision@1' : precision_at_k(test_data, df_gru_predictions, col_user="user_id", col_item="anime_id", col_rating='score', k=1),
    'Precision@10' : precision_at_k(test_data, df_gru_predictions, col_user="user_id", col_item="anime_id", col_rating='score', k=10),
    'Precision@20' : precision_at_k(test_data, df_gru_predictions, col_user="user_id", col_item="anime_id", col_rating='score', k=20),
    'Recall@1' : recall_at_k(test_data, df_gru_predictions, col_user="user_id", col_item="anime_id", col_rating='score', k=1),
    'Recall@10' : recall_at_k(test_data, df_gru_predictions, col_user="user_id", col_item="anime_id", col_rating='score', k=10),
    'Recall@20' : recall_at_k(test_data, df_gru_predictions, col_user="user_id", col_item="anime_id", col_rating='score', k=20),
    'NDCG@1' : ndcg_at_k(test_data, df_gru_predictions, col_user="user_id", col_item="anime_id", col_rating='score', k=1),
    'NDCG@10' : ndcg_at_k(test_data, df_gru_predictions, col_user="user_id", col_item="anime_id", col_rating='score', k=10),
    'NDCG@20' : ndcg_at_k(test_data, df_gru_predictions, col_user="user_id", col_item="anime_id", col_rating='score', k=20)
}

In [ ]:
pd.DataFrame({"GRU": ranking_metrics_gru})

,GRU
Precision@1,0.253500
Precision@10,0.182010
Precision@20,0.159170
Recall@1,0.015675
Recall@10,0.102082
Recall@20,0.169177
NDCG@1,0.253500
NDCG@10,0.204065
NDCG@20,0.209069


In [ ]:
test_data['user_id'] = test_data['user_id'].astype(str)
test_data['anime_id'] = test_data['anime_id'].astype(str)

df_gru_predictions['user_id'] = df_gru_predictions['user_id'].astype(str)
df_gru_predictions['anime_id'] = df_gru_predictions['anime_id'].astype(str)
df_gru_predictions['prediction'] = df_gru_predictions['prediction'].astype(float)

In [ ]:
qrels = Qrels.from_df(
    df=test_data,
    q_id_col="user_id",
    doc_id_col="anime_id",
    score_col="score",
)

run = Run.from_df(
    df=df_gru_predictions,
    q_id_col="user_id",
    doc_id_col="anime_id",
    score_col="prediction",
)

In [ ]:
ranking_metrics_gru = pd.DataFrame({'GRU': evaluate(qrels, run, ["mrr", "ndcg@10", "recall@10", "precision@10", "hit_rate@10"])})

In [ ]:
ranking_metrics_gru

,GRU
mrr,0.414703
ndcg@10,0.186208
recall@10,0.102082
precision@10,0.182010
hit_rate@10,0.789400


In [ ]:
gru.save("/content/drive/MyDrive/Final Project/Codes/animelist-goodbooks-recommendation/animelist/model")

GRU4Rec model is saved to /content/drive/MyDrive/Final Project/Codes/animelist-goodbooks-recommendation/animelist/model/GRU4Rec/2025-04-14_08-11-17-303979.pkl


'/content/drive/MyDrive/Final Project/Codes/animelist-goodbooks-recommendation/animelist/model/GRU4Rec/2025-04-14_08-11-17-303979.pkl'

## Hyperparameter Model

In [ ]:
 gru = cornac.models.GRU4Rec(
            layers=[200],
            loss='cross-entropy',
            learning_rate=0.0244,
            dropout_p_embed=0.4333,
            dropout_p_hidden=0.4606,
            batch_size=128,
            n_epochs=20,
            device="cuda",
            verbose=True,
            seed=SEED,
        )

In [ ]:
with Timer() as t:
    gru.fit(next_item_eval.train_set)
print("Took {} seconds for training.".format(t))

  0%|          | 0/20 [00:00<?, ?it/s]

Took 571.8146 seconds for training.


In [ ]:
gru.transform(next_item_eval.test_set)

In [ ]:
gru_ranking, gru_scores = ranking_eval(
    gru,
    next_item_eval.train_set,
    next_item_eval.test_set,
    exclude_unknowns=True,
    mode="last",
    verbose=False,
)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
users = []
item = list(gru.train_set.iid_map.keys())
for uid, user_idx in gru.train_set.sid_map.items():
    user = [uid] * len(item)
    users.extend(user)

In [ ]:
import itertools

flattened_ranking = list(itertools.chain(*gru_ranking))
flattened_score= list(itertools.chain(*gru_scores))

In [ ]:
len(users), len(flattened_ranking), len(flattened_score)

(10000000, 10000000, 10000000)

In [ ]:
df_gru_predictions = pd.DataFrame({'user_id':users, 'anime_id':flattened_ranking, 'prediction': flattened_score})

In [19]:
PREDICTION_PATH = '/content/drive/MyDrive/Final Project/Codes/animelist-goodbooks-recommendation/animelist/predictions'
pickle.dump(df_gru_predictions, open(f'{PREDICTION_PATH}/predictions_gru_opt.pkl', 'wb'))

In [20]:
filtered_df_gru = df_gru_predictions.merge(train_data, on=['user_id', 'anime_id'], how='left', indicator=True)
filtered_df_gru = filtered_df_gru[filtered_df_gru['_merge'] == 'left_only']
filtered_df_gru = filtered_df_gru.drop(columns=['_merge'])

In [21]:
test_data['user_id'] = test_data['user_id'].astype(str)
test_data['anime_id'] = test_data['anime_id'].astype(str)

filtered_df_gru['user_id'] = filtered_df_gru['user_id'].astype(str)
filtered_df_gru['anime_id'] = filtered_df_gru['anime_id'].astype(str)
filtered_df_gru['prediction'] = filtered_df_gru['prediction'].astype(float)

In [23]:
qrels = Qrels.from_df(
    df=test_data,
    q_id_col="user_id",
    doc_id_col="anime_id",
    score_col="score",
)

run = Run.from_df(
    df=filtered_df_gru,
    q_id_col="user_id",
    doc_id_col="anime_id",
    score_col="prediction",
)

In [24]:
ranking_metrics_gru = pd.DataFrame({'GRU': evaluate(qrels, run, ["mrr", "ndcg@10", "recall@10", "precision@10", "hit_rate@10"])})

/usr/local/lib/python3.11/dist-packages/ranx/metrics/reciprocal_rank.py:29: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  scores[i] = _reciprocal_rank(qrels[i], run[i], k, rel_lvl)


In [25]:
ranking_metrics_gru

,GRU
mrr,0.586852
ndcg@10,0.284608
recall@10,0.160245
precision@10,0.265480
hit_rate@10,0.960000


In [ ]:
gru.save("/content/drive/MyDrive/Final Project/Codes/animelist-goodbooks-recommendation/animelist/model")

GRU4Rec model is saved to /content/drive/MyDrive/Final Project/Codes/animelist-goodbooks-recommendation/animelist/model/GRU4Rec/2025-04-18_01-09-32-651532.pkl


'/content/drive/MyDrive/Final Project/Codes/animelist-goodbooks-recommendation/animelist/model/GRU4Rec/2025-04-18_01-09-32-651532.pkl'